In [1]:
import datetime
from ib_insync import *
import pandas as pd

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import VRP
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
except:
    from strategy_manager.strategies.short_vix import VRP
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager

In [2]:
ib = connect_to_IB()

# Develop a method to retrieve the current portfolio as dataframe and store it in arcticdb

In [3]:
pm = PortfolioManager(ib)

In [4]:
pm.fetch_portfolio()

/Users/johanneslange/Desktop/Pythonprojekte/IB-Multi-Strategy-ATS/broker/portfolio.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([portfolio_df, cash_df], ignore_index=True)


,symbol,contractType,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,conId,lastTradeDate,account
0,EWT,OPT,-2.0000,1.441401,-288.2800,144.317150,0.35,0.0,514008709,20240119,U7706434
1,IAU,OPT,-1.0000,0.570311,-57.0300,63.007800,5.98,0.0,584813329,20240119,U7706434
2,IUSQ,STK,100.0000,68.239998,6824.0000,65.602785,263.72,0.0,239363241,,U7706434
3,IWM,OPT,-1.0000,0.505823,-50.5800,105.627500,55.05,0.0,671118912,20231226,U7706434
4,MGC,FOP,-1.0000,0.496071,-4.9600,120.980000,116.02,0.0,657722358,20231226,U7706434
5,TLT,OPT,-2.0000,0.183240,-36.6500,38.318000,39.99,0.0,671127312,20231227,U7706434
6,TLT,OPT,-2.0000,0.127882,-25.5800,28.305600,31.03,0.0,629057662,20231229,U7706434
7,VXM,FUT,-1.0000,15.120000,-1512.0000,1569.525000,57.53,0.0,644107700,20240117,U7706434
8,XLU,OPT,-1.0000,0.334109,-33.4100,74.957700,41.55,0.0,667770698,20240105,U7706434
9,EUR,CASH,53070.6500,NaN,53070.6500,NaN,NaN,NaN,None,None,All


In [5]:
positions = [
    {  
        "symbol": item.contract.symbol,
        "contractType": item.contract.secType,
        "position": item.position,
        "marketPrice": item.marketPrice,
        "marketValue": item.marketValue,
        "averageCost": item.averageCost,
        "unrealizedPNL": item.unrealizedPNL,
        "conId": item.contract.conId,
        "lastTradeDate": item.contract.lastTradeDateOrContractMonth,
        "account": item.account
    }
    for item in ib.portfolio()
]
df = pd.DataFrame(positions)
df

,symbol,contractType,position,marketPrice,marketValue,averageCost,unrealizedPNL,conId,lastTradeDate,account
0,EWT,OPT,-2.0,1.441401,-288.28,144.317150,0.35,514008709,20240119,U7706434
1,IAU,OPT,-1.0,0.570311,-57.03,63.007800,5.98,584813329,20240119,U7706434
2,IUSQ,STK,100.0,68.239998,6824.00,65.602785,263.72,239363241,,U7706434
3,IWM,OPT,-1.0,0.505823,-50.58,105.627500,55.05,671118912,20231226,U7706434
4,MGC,FOP,-1.0,0.496071,-4.96,120.980000,116.02,657722358,20231226,U7706434
5,TLT,OPT,-2.0,0.183240,-36.65,38.318000,39.99,671127312,20231227,U7706434
6,TLT,OPT,-2.0,0.127882,-25.58,28.305600,31.03,629057662,20231229,U7706434
7,VXM,FUT,-1.0,15.120000,-1512.00,1569.525000,57.53,644107700,20240117,U7706434
8,XLU,OPT,-1.0,0.334109,-33.41,74.957700,41.55,667770698,20240105,U7706434


In [6]:
# Fetch cash positions
# cash_positions = [item for item in ib.accountSummary() if item.tag == 'CashBalance']

# Convert cash positions to a list of dictionaries
cash_positions = [
    {
        "symbol": item.currency,
        "contractType": "CASH",
        "position": float(item.value),
        "marketPrice": None,
        "marketValue": float(item.value),
        "averageCost": None,
        "unrealizedPNL": None,
        "realizedPNL": None,
        "conId": None,
        "lastTradeDate": None,
        "account": item.account
    }
    for item in ib.accountSummary() if item.tag == 'CashBalance'
]

cash_df = pd.DataFrame(cash_positions)
cash_df

,symbol,contractType,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,conId,lastTradeDate,account
0,EUR,CASH,53070.6500,None,53070.6500,None,None,None,None,None,All
1,USD,CASH,44822.5500,None,44822.5500,None,None,None,None,None,All
2,BASE,CASH,93770.3231,None,93770.3231,None,None,None,None,None,All


In [5]:
cash_positions = [item for item in ib.accountSummary() if item.tag=='CashBalance' and item.currency != "BASE"]

In [6]:
cash_positions

[AccountValue(account='All', tag='CashBalance', value='53070.65', currency='EUR', modelCode=''),
 AccountValue(account='All', tag='CashBalance', value='44822.55', currency='USD', modelCode='')]

In [25]:
cash_positions = [item for item in ib.accountSummary() if item.tag == 'CashBalance' and item.currency != "BASE"]
cash_data = [
    {
        "symbol": item.currency,
        "contractType": "CASH",
        "position": float(item.value),
        "marketPrice": None,
        "marketValue": float(item.value),
        "averageCost": None,
        "unrealizedPNL": None,
        "realizedPNL": None,
        "conId": None,
        "lastTradeDate": None,
        "account": item.account
    }
    for item in cash_positions
]


base = [{"symbol": "TOTAL_CASH",
        "contractType": "CASH",
        "position": float(item.value),
        "marketPrice": None,
        "marketValue": float(item.value),
        "averageCost": None,
        "unrealizedPNL": None,
        "realizedPNL": None,
        "conId": None,
        "lastTradeDate": None,
        "account": item.account} 
        for item in ib.accountSummary() if item.currency == "BASE" and item.tag == 'CashBalance']

cash_data.append(base[0])

cash_df = pd.DataFrame(cash_data)
cash_df

,symbol,contractType,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,conId,lastTradeDate,account
0,EUR,CASH,53070.6500,None,53070.6500,None,None,None,None,None,All
1,USD,CASH,44822.5500,None,44822.5500,None,None,None,None,None,All
2,TOTAL_CASH,CASH,93770.3231,None,93770.3231,None,None,None,None,None,All


In [20]:
cash_df

,symbol,contractType,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,conId,lastTradeDate,account
0,EUR,CASH,53070.65,None,53070.65,None,None,None,None,None,All
1,USD,CASH,44822.55,None,44822.55,None,None,None,None,None,All
2,EUR,CASH,53070.65,None,53070.65,None,None,None,None,None,All


In [10]:
vrp = VRP(ib,TradeManager)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

/Users/johanneslange/Desktop/Pythonprojekte/IB-Multi-Strategy-ATS/strategy_manager/strategies/short_vix.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_df['Realised Volatility'] = spx_df['Return'].rolling(21).std()*np.sqrt(252)*100
Error 200, reqId 75: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202312')
Unknown contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202312')



Latest Volatility Risk Premium                   Close    Return  Realised Volatility    VIX       VRP
Date                                                                   
2023-12-18  4740.560059  0.004528             6.971769  12.56  7.348231
2023-12-19  4768.370117  0.005866             7.059069  12.53  6.740931
2023-12-20  4698.350098 -0.014684             9.043557  13.67  4.366442
2023-12-21  4746.750000  0.010301             9.426038  13.65  3.923962
2023-12-22  4754.629883  0.001660             9.401645  13.03  3.448356
Current Spot Rate: 13.029999732971191
[Future(conId=644107700, symbol='VXM', lastTradeDateOrContractMonth='20240117', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMF4', tradingClass='VXM')]: 15.1


Error 354, reqId 81: Dieses aufgerufene Marktdaten Packet haben Sie nicht abonniert.VXM MAR'24/TOP/ALL, contract: Future(conId=656210801, symbol='VXM', lastTradeDateOrContractMonth='20240320', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMH4', tradingClass='VXM')


[Future(conId=649526843, symbol='VXM', lastTradeDateOrContractMonth='20240214', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMG4', tradingClass='VXM')]: 16.3


...

[Future(conId=661277712, symbol='VXM', lastTradeDateOrContractMonth='20240417', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMJ4', tradingClass='VXM')]: nan


Error 354, reqId 87: Dieses aufgerufene Marktdaten Packet haben Sie nicht abonniert.VXM JUN'24/TOP/ALL, contract: Future(conId=674402981, symbol='VXM', lastTradeDateOrContractMonth='20240618', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMM4', tradingClass='VXM')


[Future(conId=666994095, symbol='VXM', lastTradeDateOrContractMonth='20240522', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMK4', tradingClass='VXM')]: nan


Error 200, reqId 88: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202407')
Unknown contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202407')
Error 200, reqId 89: Es wurde keine Wertpapierdefinition zu der Anfrage gefunden, contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202408')
Unknown contract: Future(symbol='VXM', lastTradeDateOrContractMonth='202408')


[Future(conId=674402981, symbol='VXM', lastTradeDateOrContractMonth='20240618', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMM4', tradingClass='VXM')]: nan


In [11]:
cc = Future(localSymbol="VXMF4")
nc = Future(localSymbol="VXMG4")
ib.qualifyContracts(cc)
ib.qualifyContracts(nc)

[Future(conId=649526843, symbol='VXM', lastTradeDateOrContractMonth='20240214', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMG4', tradingClass='VXM')]

In [12]:
vrp.TradeManager.roll_future(vrp,cc,nc,"test2")

Trade(contract=Contract(secType='BAG', symbol='VXM', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=644107700, ratio=1, action='BUY', exchange='CFE', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=649526843, ratio=1, action='SELL', exchange='CFE', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=MarketOrder(orderId=94, action='BUY', totalQuantity=1.0, orderRef='test2'), orderStatus=OrderStatus(orderId=94, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2023, 12, 23, 23, 14, 59, 387932, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

[Future(conId=649526843, symbol='VXM', lastTradeDateOrContractMonth='20240214', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMG4', tradingClass='VXM')]

In [12]:
next_f = ib.reqMktData(nc)

In [13]:
next_f

Ticker(contract=Future(conId=649526843, symbol='VXM', lastTradeDateOrContractMonth='20240214', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMG4', tradingClass='VXM'), time=datetime.datetime(2023, 12, 23, 22, 42, 28, 671695, tzinfo=datetime.timezone.utc), marketDataType=4, minTick=0.01, bid=16.29, bidSize=2.0, ask=16.3, askSize=6.0, last=16.3, lastSize=1.0, volume=895.0, high=16.73, low=16.3, close=16.52, ticks=[TickData(time=datetime.datetime(2023, 12, 23, 22, 42, 28, 671695, tzinfo=datetime.timezone.utc), tickType=68, price=16.3, size=1.0), TickData(time=datetime.datetime(2023, 12, 23, 22, 42, 28, 671695, tzinfo=datetime.timezone.utc), tickType=72, price=16.73, size=0.0), TickData(time=datetime.datetime(2023, 12, 23, 22, 42, 28, 671695, tzinfo=datetime.timezone.utc), tickType=73, price=16.3, size=0.0), TickData(time=datetime.datetime(2023, 12, 23, 22, 42, 28, 671695, tzinfo=datetime.timezone.utc), tickType=74, price=-1.0, size=895.0), TickData(time=datetime.datetim

In [27]:
def roll_future(self, current_contract, new_contract, orderRef=""):
    # Qualify contracts
    self.ib.qualifyContracts(current_contract,new_contract)
    ib.sleep(1)
    print(new_contract)
    # Define quantity based on current position
    quantity = [pos.position for pos in self.ib.portfolio() if pos.contract.localSymbol==current_contract.localSymbol][0]

    # Define the combination contract (bag)
    bag = Contract(symbol=current_contract.symbol, secType='BAG', exchange='SMART', currency=current_contract.currency)
    bag.comboLegs = [
        ComboLeg(conId=current_contract.conId, ratio=1, action="SELL" if quantity > 0 else "BUY",exchange='CFE'),  # Exiting the current contract
        ComboLeg(conId=new_contract.conId, ratio=1, action="BUY" if quantity > 0 else "SELL",exchange='CFE')        # Entering the new contract
    ]

    # Create the order - here we use a Market order as an example
    order = MarketOrder('BUY', abs(quantity))
    order.orderRef = orderRef

    # Place the order
    trade = self.ib.placeOrder(bag, order)
    return trade

In [28]:
roll_future(vrp,cc,nc,"test")

Future(conId=649526843, symbol='VXM', lastTradeDateOrContractMonth='20240214', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMG4', tradingClass='VXM')


Trade(contract=Contract(secType='BAG', symbol='VXM', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=644107700, ratio=1, action='BUY', exchange='CFE', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=649526843, ratio=1, action='SELL', exchange='CFE', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=MarketOrder(orderId=48, action='BUY', totalQuantity=1.0, orderRef='test'), orderStatus=OrderStatus(orderId=48, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2023, 12, 23, 23, 7, 4, 775502, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

Error 161, reqId 48: Es wurde eine Stornierung versucht, die Order ist jedoch nicht in einem stornierbaren Zustand. Order permId =154751771
Canceled order: Trade(contract=Contract(secType='BAG', symbol='VXM', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=644107700, ratio=1, action='BUY', exchange='CFE', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=649526843, ratio=1, action='SELL', exchange='CFE', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=MarketOrder(orderId=48, permId=154751771, action='BUY', totalQuantity=1.0, lmtPrice=0.0, auxPrice=0.0, orderRef='test'), orderStatus=OrderStatus(orderId=48, status='Cancelled', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=154751771, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2023, 12, 23, 23, 7, 4, 775502, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', error

In [24]:
ib.qualifyContracts(Future(conId="649526843"))

[Future(conId=649526843, symbol='VXM', lastTradeDateOrContractMonth='20240214', multiplier='100', exchange='CFE', currency='USD', localSymbol='VXMG4', tradingClass='VXM')]